# Sparkathon 30.01.2018

## Recommender system

Najpierw przetestujemy środowisko

In [1]:
spark

org.apache.spark.sql.SparkSession@1f36f06f

### Wczytywanie danych

Wczytamy plik. opcja header oznacza że pierwsza linijka zawiera nazwy kolumn
inferSchema oznacza że Spark automatycznie spróbuje wywnioskować jaki kształ mają dane wejściowe i na tej podstawie zbuduje DataFrame

#### Ratings (Movie lens dataset)

In [21]:
val ratings = spark.read.option("header", true).
                        option("inferSchema", true).
                        csv("/home/jovyan/resources/small/ratings.csv")

#### Movies (Movielens dataset) 
tu się znajdują informację o filmach (nazwa, rok, gatunek i oczywiście id)

In [22]:
val movies = spark.read.option("header", true).
                        option("inferSchema", true).
                        csv("/home/jovyan/resources/small/movies.csv")

In [23]:
ratings.show

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



In [24]:
movies.show

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

#### Dzielimy dane na zbiór treningowy i zbiór testowy

In [25]:
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))

In [26]:
training.show

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   3671|   3.0|1260759117|
|     2|     10|   4.0| 835355493|
|     2|     17|   5.0| 835355681|
|     2|     39|   5.0| 835355604|
|     2|     47|   4.0| 835355552|
+------+-------+------+----------+
only showing top 20 rows



In [27]:
test.show

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|   1061|   3.0|1260759182|
|     1|   1405|   1.0|1260759203|
|     1|   2150|   3.0|1260759194|
|     1|   2968|   1.0|1260759200|
|     2|    235|   3.0| 835355664|
|     2|    300|   3.0| 835355532|
|     2|    356|   3.0| 835355628|
|     2|    357|   3.0| 835355749|
|     2|    364|   3.0| 835355604|
|     2|    500|   4.0| 835355731|
|     2|    509|   4.0| 835355719|
|     2|    539|   3.0| 835355767|
|     2|    550|   3.0| 835356109|
|     2|    551|   5.0| 835355767|
|     2|    552|   3.0| 835355860|
|     2|    588|   3.0| 835355441|
|     2|    589|   5.0| 835355697|
|     2|    592|   5.0| 835355395|
|     2|    661|   4.0| 835356141|
|     2|    720|   4.0| 835355978|
+------+-------+------+----------+
only showing top 20 rows



#### Importujemy implementację Sparkową algorytmu Alternating Least Squares (ALS)

In [28]:
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
 
val als = new ALS().setMaxIter(5).
                    setRegParam(0.01).
                    setColdStartStrategy("drop").
                    setUserCol("userId").
                    setItemCol("movieId").
                    setRatingCol("rating")

#### Tworzymy model z danych wejściowych (dataframu ratings) i algorytmu ALS

In [29]:
val model: ALSModel = als.fit(training)

In [31]:
model.recommendForAllUsers(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[1940,5.7621803]...|
|   463|[[3167,5.303713],...|
|   496|[[3213,9.252211],...|
|   148|[[3672,7.3053994]...|
|   540|[[1590,11.752739]...|
|   392|[[2936,6.7001123]...|
|   243|[[2202,5.209426],...|
|   623|[[390,6.96364], [...|
|    31|[[121231,5.961924...|
|   516|[[6413,6.7407246]...|
|   580|[[1303,5.1315165]...|
|   251|[[3606,6.603481],...|
|   451|[[72,7.476898], [...|
|    85|[[42418,10.92686]...|
|   137|[[429,8.476043], ...|
|    65|[[2929,7.932242],...|
|   458|[[65188,6.6728544...|
|   481|[[1590,5.8115954]...|
|    53|[[4965,9.05489], ...|
|   255|[[25850,6.2722964...|
+------+--------------------+
only showing top 20 rows



### Ocena wyniku

Dla oceny wyniku musimy najpierw przekształcić zbiór testowy za pomocą naszego modełu.

In [32]:
val predictions = model.transform(test)

I stworzyć metodę oceny wyniku. Wybrałem RMSE (Root mean square error) jako miara błędu.

In [34]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

val evaluator = new RegressionEvaluator().setMetricName("rmse").
                                            setLabelCol("rating").
                                            setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

Root-mean-square error = 1.1237248533839757
